
# Analyze Data Quality with SageMaker Processing Jobs and Spark

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Spark are used to process and analyze data sets in order to detect data quality issues and prepare them for model training.  

In this notebook we'll use Amazon SageMaker Processing with a library called [**Deequ**](https://github.com/awslabs/deequ), and leverage the power of Spark with a managed SageMaker Processing Job to run our data processing workloads.

Here are some great resources on Deequ: 
* Blog Post:  https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/
* Research Paper:  https://assets.amazon.science/4a/75/57047bd343fabc46ec14b34cdb3b/towards-automated-data-quality-management-for-machine-learning.pdf

![Deequ](img/deequ.png)

![](img/processing.jpg)

# Amazon Customer Reviews Dataset

https://s3.amazonaws.com/amazon-reviews-pds/readme.html

### Dataset Columns:

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.

In [1]:
%store -r ingest_create_athena_table_tsv_passed

In [2]:
try:
    ingest_create_athena_table_tsv_passed
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN THE NOTEBOOKS IN THE INGEST FOLDER FIRST. You did not register the TSV Data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')

In [3]:
print(ingest_create_athena_table_tsv_passed)

True


In [4]:
if not ingest_create_athena_table_tsv_passed:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN THE NOTEBOOKS IN THE INGEST FOLDER FIRST. You did not register the TSV Data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')

[OK]


# Run the Analysis Job using a SageMaker Processing Job with Spark
Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built with our Spark script.

In [5]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

# Review the Spark preprocessing script.

In [6]:
!pygmentize preprocess-deequ.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

def main():
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))
    
    # Retrieve the args and replace 's3://' with 's3a://' (used by Spark)
    s3_input_data = args['s3_input_data'].replace('s3://', 's3a://')
    print(s3_input_data)
    s3_output_analyze_data = args['s3_output_analyze_data'].replace('s3://', 's3a://')
    print(s3_output_analyze_data)
    
    spark = SparkSession.builder \
        .appName("Amazon_Reviews_Spark_Analyzer") \
        .getOrCreate()

    # Invoke Main from preprocess-deequ.jar
    getattr(spark._jvm.SparkAmazonReviewsAnalyzer, "run")(s3_input_data, s3_output_analyze_data)

if __name__ == "__main__":
    main()


In [7]:
!pygmentize preprocess-deequ.scala

import com.amazon.deequ.analyzers.runners.{AnalysisRunner, AnalyzerContext}
import com.amazon.deequ.analyzers.runners.AnalyzerContext.successMetricsAsDataFrame
import com.amazon.deequ.analyzers.{Compliance, Correlation, Size, Completeness, Mean, ApproxCountDistinct}
import com.amazon.deequ.{VerificationSuite, VerificationResult}
import com.amazon.deequ.VerificationResult.checkResultsAsDataFrame
import com.amazon.deequ.checks.{Check, CheckLevel}
import com.amazon.deequ.suggestions.{ConstraintSuggestionRunner, Rules}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType}


object SparkAmazonReviewsAnalyzer {
  def run(s3InputData: String, s3OutputAnalyzeData: String): Unit = {

    System.out.println(s"s3_input_data: ${s3InputData}")
    System.out.println(s"s3_output_analyze_data: ${s3OutputAnalyzeData}")
      
    val spark = SparkSession
      .builder
      .appName("SparkAmaz

In [8]:
from sagemaker.spark.processing import PySparkProcessor

processor = PySparkProcessor(base_job_name='spark-amazon-reviews-analyzer',
                            role=role,
                            instance_count=1,
                            instance_type='ml.r5.2xlarge',
                            max_runtime_in_seconds=1200)

In [9]:
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-west-2-085964654406/amazon-reviews-pds/tsv/


In [10]:
!aws s3 ls $s3_input_data

2020-09-26 16:39:04   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-09-26 16:39:08   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


## Setup Output Data

In [11]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)
processing_job_name = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)

print('Processing job name:  {}'.format(processing_job_name))

Processing job name:  amazon-reviews-spark-analyzer-2020-09-26-17-25-06


In [12]:
s3_output_analyze_data = 's3://{}/{}/output'.format(bucket, output_prefix)

print(s3_output_analyze_data)

s3://sagemaker-us-west-2-085964654406/amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output


## Start the Spark Processing Job

_Notes on Invoking from Lambda:_
* However, if we use the boto3 SDK (ie. with a Lambda), we need to copy the `preprocess.py` file to S3 and specify the everything include --py-files, etc.
* We would need to do the following before invoking the Lambda:
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/code/preprocess.py
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/py_files/preprocess.py
* Then reference the s3://<location> above in the --py-files, etc.
* See Lambda example code in this same project for more details.

_Notes on not using ProcessingInput and Output:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by ProcessingInput and ProcessingOutput (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes._"
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use ProcessingInput, the data will be copied to each node (which we don't want in this case since Spark already handles this)

In [13]:
from sagemaker.processing import ProcessingOutput

processor.run(submit_app='preprocess-deequ.py',
              submit_jars=['deequ-1.0.1.jar', 'preprocess-deequ.jar'],
              arguments=['s3_input_data', s3_input_data,
                         's3_output_analyze_data', s3_output_analyze_data,
              ],
              logs=True,
              wait=False
)


Job Name:  spark-amazon-reviews-analyzer-2020-09-26-17-25-06-264
Inputs:  [{'InputName': 'jars', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-085964654406/spark-amazon-reviews-analyzer-2020-09-26-17-25-06-264/input/jars', 'LocalPath': '/opt/ml/processing/input/jars', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-085964654406/spark-amazon-reviews-analyzer-2020-09-26-17-25-06-264/input/code/preprocess-deequ.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  []


In [14]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(region, processing_job_name)))


In [15]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After a Few Minutes</b>'.format(region, processing_job_name)))


In [16]:
from IPython.core.display import display, HTML

s3_job_output_prefix = output_prefix

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, s3_job_output_prefix, region)))


# Monitor the Processing Job

In [17]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

print(processing_job_description)

{'ProcessingInputs': [{'InputName': 'jars', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-085964654406/spark-amazon-reviews-analyzer-2020-09-26-17-25-06-264/input/jars', 'LocalPath': '/opt/ml/processing/input/jars', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-085964654406/spark-amazon-reviews-analyzer-2020-09-26-17-25-06-264/input/code/preprocess-deequ.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingJobName': 'spark-amazon-reviews-analyzer-2020-09-26-17-25-06-264', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1, 'InstanceType': 'ml.r5.2xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 1200}, 'AppSpecification': {'ImageUri': '153931337802.dkr.ecr.us-west-2.ama

In [18]:
running_processor.wait()

........................09-26 17:28 smspark.cli  INFO     Parsing arguments. argv: ['/usr/local/bin/smspark-submit', '--jars', '/opt/ml/processing/input/jars', '/opt/ml/processing/input/code/preprocess-deequ.py', 's3_input_data', 's3://sagemaker-us-west-2-085964654406/amazon-reviews-pds/tsv/', 's3_output_analyze_data', 's3://sagemaker-us-west-2-085964654406/amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output']
09-26 17:28 smspark.cli  INFO     Raw spark options before processing: {'jars': '/opt/ml/processing/input/jars', 'class_': None, 'py_files': None, 'files': None, 'verbose': False}
09-26 17:28 smspark.cli  INFO     App and app arguments: ['/opt/ml/processing/input/code/preprocess-deequ.py', 's3_input_data', 's3://sagemaker-us-west-2-085964654406/amazon-reviews-pds/tsv/', 's3_output_analyze_data', 's3://sagemaker-us-west-2-085964654406/amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output']
09-26 17:28 smspark.cli  INFO     Rendered spark options: {'jars': '/opt/ml/processi

20/09/26 17:29:04 INFO hdfs.StateChange: BLOCK* allocate blk_1073741825_1001, replicas=10.0.198.240:50010 for /user/root/.sparkStaging/application_1601141334775_0001/__spark_libs__1580634755884742823.zip
20/09/26 17:29:05 INFO datanode.DataNode: Receiving BP-600261174-10.0.198.240-1601141332745:blk_1073741825_1001 src: /10.0.198.240:39938 dest: /10.0.198.240:50010
20/09/26 17:29:06 INFO DataNode.clienttrace: src: /10.0.198.240:39938, dest: /10.0.198.240:50010, bytes: 134217728, op: HDFS_WRITE, cliID: DFSClient_NONMAPREDUCE_-1583662445_19, offset: 0, srvID: a031eaf1-9acd-417e-ace8-159204f4e8ba, blockid: BP-600261174-10.0.198.240-1601141332745:blk_1073741825_1001, duration: 233336238
20/09/26 17:29:06 INFO datanode.DataNode: PacketResponder: BP-600261174-10.0.198.240-1601141332745:blk_1073741825_1001, type=LAST_IN_PIPELINE terminating
20/09/26 17:29:06 INFO hdfs.StateChange: BLOCK* allocate blk_1073741826_1002, replicas=10.0.198.240:50010 for /user/root/.sparkStaging/application_16011413

20/09/26 17:29:14 INFO yarn.Client: Application report for application_1601141334775_0001 (state: ACCEPTED)
20/09/26 17:29:14 INFO ipc.Server: Auth successful for appattempt_1601141334775_0001_000001 (auth:SIMPLE)
20/09/26 17:29:14 INFO resourcemanager.ApplicationMasterService: AM registration appattempt_1601141334775_0001_000001
20/09/26 17:29:14 INFO resourcemanager.RMAuditLogger: USER=root#011IP=10.0.198.240#011OPERATION=Register App Master#011TARGET=ApplicationMasterService#011RESULT=SUCCESS#011APPID=application_1601141334775_0001#011APPATTEMPTID=appattempt_1601141334775_0001_000001
20/09/26 17:29:14 INFO attempt.RMAppAttemptImpl: appattempt_1601141334775_0001_000001 State change from LAUNCHED to RUNNING
20/09/26 17:29:14 INFO rmapp.RMAppImpl: application_1601141334775_0001 State change from ACCEPTED to RUNNING on event=ATTEMPT_REGISTERED
20/09/26 17:29:15 INFO cluster.YarnClientSchedulerBackend: Add WebUI Filter. org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter, Map(PROX

20/09/26 17:29:24 INFO scheduler.TaskSetManager: Finished task 1.0 in stage 0.0 (TID 1) in 3475 ms on algo-1 (executor 1) (1/2)
20/09/26 17:29:24 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 3734 ms on algo-1 (executor 1) (2/2)
20/09/26 17:29:24 INFO cluster.YarnScheduler: Removed TaskSet 0.0, whose tasks have all completed, from pool 
20/09/26 17:29:24 INFO scheduler.DAGScheduler: ShuffleMapStage 0 (collect at AnalysisRunner.scala:303) finished in 3.784 s
20/09/26 17:29:24 INFO scheduler.DAGScheduler: looking for newly runnable stages
20/09/26 17:29:24 INFO scheduler.DAGScheduler: running: Set()
20/09/26 17:29:24 INFO scheduler.DAGScheduler: waiting: Set()
20/09/26 17:29:24 INFO scheduler.DAGScheduler: failed: Set()
20/09/26 17:29:24 INFO adaptive.CoalesceShufflePartitions: advisoryTargetPostShuffleInputSize: 67108864, targetPostShuffleInputSize 48.
20/09/26 17:29:24 INFO spark.SparkContext: Starting job: collect at AnalysisRunner.scala:303
20/09/26 17:29:2

20/09/26 17:29:34 INFO scheduler.TaskSetManager: Starting task 22.0 in stage 14.0 (TID 41, algo-1, executor 1, partition 22, PROCESS_LOCAL, 7767 bytes)
20/09/26 17:29:34 INFO scheduler.TaskSetManager: Finished task 14.0 in stage 14.0 (TID 33) in 96 ms on algo-1 (executor 1) (15/26)
20/09/26 17:29:34 INFO scheduler.TaskSetManager: Starting task 23.0 in stage 14.0 (TID 42, algo-1, executor 1, partition 23, PROCESS_LOCAL, 7767 bytes)
20/09/26 17:29:34 INFO scheduler.TaskSetManager: Finished task 15.0 in stage 14.0 (TID 34) in 74 ms on algo-1 (executor 1) (16/26)
20/09/26 17:29:34 INFO scheduler.TaskSetManager: Starting task 24.0 in stage 14.0 (TID 43, algo-1, executor 1, partition 24, PROCESS_LOCAL, 7767 bytes)
20/09/26 17:29:34 INFO scheduler.TaskSetManager: Finished task 16.0 in stage 14.0 (TID 35) in 57 ms on algo-1 (executor 1) (17/26)
20/09/26 17:29:34 INFO scheduler.TaskSetManager: Starting task 25.0 in stage 14.0 (TID 44, algo-1, executor 1, partition 25, PROCESS_LOCAL, 7767 bytes)

[/var/log/yarn/userlogs/application_1601141334775_0001/container_1601141334775_0001_01_000002/stderr] 20/09/26 17:29:42 INFO executor.Executor: Finished task 1.0 in stage 24.0 (TID 63). 2386 bytes result sent to driver
[/var/log/yarn/userlogs/application_1601141334775_0001/container_1601141334775_0001_01_000002/stderr] 20/09/26 17:29:43 INFO executor.Executor: Finished task 0.0 in stage 24.0 (TID 62). 2386 bytes result sent to driver
[/var/log/yarn/userlogs/application_1601141334775_0001/container_1601141334775_0001_01_000002/stderr] 20/09/26 17:29:43 INFO executor.CoarseGrainedExecutorBackend: Got assigned task 64
[/var/log/yarn/userlogs/application_1601141334775_0001/container_1601141334775_0001_01_000002/stderr] 20/09/26 17:29:43 INFO executor.Executor: Running task 0.0 in stage 26.0 (TID 64)
[/var/log/yarn/userlogs/application_1601141334775_0001/container_1601141334775_0001_01_000002/stderr] 20/09/26 17:29:43 INFO spark.MapOutputTrackerWorker: Updating epoch to 9 and clearing cache

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# Inspect the Processed Output 

## These are the quality checks on our dataset.

## _The next cells will not work properly until the job completes above._

In [19]:
!aws s3 ls --recursive $s3_output_analyze_data/

2020-09-26 17:29:37          0 amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/constraint-checks/_SUCCESS
2020-09-26 17:29:36        768 amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/constraint-checks/part-00000-e61c9eec-33cc-49b5-b6b4-c0b96caf0b22-c000.csv
2020-09-26 17:29:52          0 amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/constraint-suggestions/_SUCCESS
2020-09-26 17:29:51       2289 amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/constraint-suggestions/part-00000-e399f995-b204-418e-966c-9839eea91a78-c000.csv
2020-09-26 17:29:29          0 amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/dataset-metrics/_SUCCESS
2020-09-26 17:29:28        364 amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/dataset-metrics/part-00000-ed82b9cd-8abe-464e-a789-444f531aa651-c000.csv
2020-09-26 17:29:39          0 amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/success-metrics/_SUCCESS
2020-09-26 17:29:38        277 amazon-reviews-s

## Copy the Output from S3 to Local
* dataset-metrics/
* constraint-checks/
* success-metrics/
* constraint-suggestions/


In [20]:
!aws s3 cp --recursive $s3_output_analyze_data ./amazon-reviews-spark-analyzer/ --exclude="*" --include="*.csv"

download: s3://sagemaker-us-west-2-085964654406/amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/dataset-metrics/part-00000-ed82b9cd-8abe-464e-a789-444f531aa651-c000.csv to amazon-reviews-spark-analyzer/dataset-metrics/part-00000-ed82b9cd-8abe-464e-a789-444f531aa651-c000.csv
download: s3://sagemaker-us-west-2-085964654406/amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/constraint-suggestions/part-00000-e399f995-b204-418e-966c-9839eea91a78-c000.csv to amazon-reviews-spark-analyzer/constraint-suggestions/part-00000-e399f995-b204-418e-966c-9839eea91a78-c000.csv
download: s3://sagemaker-us-west-2-085964654406/amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/constraint-checks/part-00000-e61c9eec-33cc-49b5-b6b4-c0b96caf0b22-c000.csv to amazon-reviews-spark-analyzer/constraint-checks/part-00000-e61c9eec-33cc-49b5-b6b4-c0b96caf0b22-c000.csv
download: s3://sagemaker-us-west-2-085964654406/amazon-reviews-spark-analyzer-2020-09-26-17-25-06/output/success-metrics/part-0000

## Analyze Constraint Checks

In [21]:
import glob
import pandas as pd
import os

def load_dataset(path, sep, header):
    data = pd.concat([pd.read_csv(f, sep=sep, header=header) for f in glob.glob('{}/*.csv'.format(path))], ignore_index = True)

    return data

In [22]:
df_constraint_checks = load_dataset(path='./amazon-reviews-spark-analyzer/constraint-checks/', sep='\t', header=0)
df_constraint_checks[['check', 'constraint', 'constraint_status', 'constraint_message']]

,check,constraint,constraint_status,constraint_message
0,Review Check,SizeConstraint(Size(None)),Success,NaN
1,Review Check,"MinimumConstraint(Minimum(star_rating,None))",Success,NaN
2,Review Check,"MaximumConstraint(Maximum(star_rating,None))",Success,NaN
3,Review Check,"CompletenessConstraint(Completeness(review_id,...",Success,NaN
4,Review Check,UniquenessConstraint(Uniqueness(List(review_id))),Success,NaN
5,Review Check,CompletenessConstraint(Completeness(marketplac...,Success,NaN
6,Review Check,ComplianceConstraint(Compliance(marketplace co...,Success,NaN


## Analyze Dataset Metrics

In [23]:
df_dataset_metrics = load_dataset(path='./amazon-reviews-spark-analyzer/dataset-metrics/', sep='\t', header=0)
df_dataset_metrics

,entity,instance,name,value
0,Column,review_id,Completeness,1.000000
1,Column,review_id,ApproxCountDistinct,238027.000000
2,Mutlicolumn,"total_votes,star_rating",Correlation,-0.080881
3,Dataset,*,Size,247515.000000
4,Column,star_rating,Mean,3.723706
5,Column,top star_rating,Compliance,0.663338
6,Mutlicolumn,"total_votes,helpful_votes",Correlation,0.980529


## Analyze Success Metrics

In [24]:
df_success_metrics = load_dataset(path='./amazon-reviews-spark-analyzer/success-metrics/', sep='\t', header=0)
df_success_metrics

,entity,instance,name,value
0,Column,review_id,Completeness,1.0
1,Column,review_id,Uniqueness,1.0
2,Dataset,*,Size,247515.0
3,Column,star_rating,Maximum,5.0
4,Column,star_rating,Minimum,1.0
5,Column,"marketplace contained in US,UK,DE,JP,FR",Compliance,1.0
6,Column,marketplace,Completeness,1.0


## Analyze Constraint Suggestions

In [25]:
df_constraint_suggestions = load_dataset(path='./amazon-reviews-spark-analyzer/constraint-suggestions/', sep='\t', header=0)
df_constraint_suggestions.columns=['column_name', 'description', 'code']
df_constraint_suggestions

,column_name,description,code
0,review_id,'review_id' is not null,".isComplete(\review_id\"")"""
1,customer_id,'customer_id' is not null,".isComplete(\customer_id\"")"""
2,customer_id,'customer_id' has type Integral,".hasDataType(\customer_id\"", ConstrainableData..."
3,customer_id,'customer_id' has no negative values,".isNonNegative(\customer_id\"")"""
4,review_date,'review_date' is not null,".isComplete(\review_date\"")"""
5,helpful_votes,'helpful_votes' is not null,".isComplete(\helpful_votes\"")"""
6,helpful_votes,'helpful_votes' has no negative values,".isNonNegative(\helpful_votes\"")"""
7,star_rating,'star_rating' is not null,".isComplete(\star_rating\"")"""
8,star_rating,'star_rating' has no negative values,".isNonNegative(\star_rating\"")"""
9,product_title,'product_title' is not null,".isComplete(\product_title\"")"""


# Save for the Next Notebook(s)

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();